# 03 - Optimisation et Expérimentation sur les Features

> Ce notebook est dédié à la **création, au test et à l’analyse de nouvelles features** pour le projet AstroSpectro.  

Il permet de comparer facilement l’impact de chaque nouvelle feature (FWHM, skewness, indices couleurs…) sur la qualité de la classification.


#

## 1. Contexte et Objectifs

- Objectif : Optimiser la performance du modèle via de nouvelles features.
- Ce notebook sert de laboratoire rapide avant migration définitive des améliorations dans le pipeline.
- **Astuce** : Documenter chaque essai, archiver code & décisions dans `logs/ideas.txt` ou `reports/idea_log.md`.



#

## 2. Imports & configuration

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

from utils import setup_project_env
from pipeline.master import MasterPipeline
from pipeline.feature_engineering import FeatureEngineer # Adapter si besoin
from pipeline.peak_detector import PeakDetector

# Initialisation automatique de l'environnement et des chemins
paths = setup_project_env()

# Instanciation du pipeline maître
pipeline = MasterPipeline(
    raw_data_dir=paths["RAW_DATA_DIR"],
    catalog_dir=paths["CATALOG_DIR"],
    processed_dir=paths["PROCESSED_DIR"],
    models_dir=paths["MODELS_DIR"],
    reports_dir=paths["REPORTS_DIR"],
)

print("\nSetup terminé. Tu es prêt à lancer ton pipeline.")

#

## 3. Chargement des données

- Charger un batch de spectres prétraités (`data/processed/`)
- Importer features, labels et spectres normalisés.

In [ ]:
# Exemple
features_df = pd.read_csv('../data/processed/features_20250802T004135Z.csv') # À adapter
labels = features_df['subclass']
X = features_df.drop(['subclass'], axis=1).select_dtypes(include=['float64', 'int64'])
print(X.columns)
print(X.head(2))

#

## 4. Exploration des features existantes

- Afficher la liste des features, leur type, leur distribution.
- Matrice de corrélation, visualisation de l’importance via RandomForest.

In [ ]:
# Aperçu features
print(features_df.info())
display(features_df.describe())
plt.figure(figsize=(10,5))
features_df.hist(bins=30, figsize=(15,10))

#

### Importance des features (baseline)

In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X, labels)
importances = pd.Series(rf.feature_importances_, index=X.columns)
importances.sort_values().plot(kind='barh', figsize=(8,6))
plt.title("Importance des features (Baseline)")
plt.show()

#

#